remove those features that are most useful keywords for distinguishing target is author or not. 
if the target is the author, then whatever differences there are between their keywords will be reflected in only a relatively small number of features, despite possible differences in theme, genre and the like.

In [1]:
import random
import copy
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import OneClassSVM
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV

In [2]:
from prince import MCA

In [2]:
# Openinng the JSON file
openTrainData = open('data/train.json');
openTestData = open('data/test.json');

# returns JSON object as
# a dictionary
trainData = json.load(openTrainData);
testData = json.load(openTestData);

# Iterating through the trainData
#for i in trainData:
#    print(trainData[i]);

# Iterating through the testData
#for i in testData:
#    print(testData[i]);

# Closing file
openTrainData.close();
openTestData.close();

len(trainData.items())

26108

In [3]:
authorIndexListDict = defaultdict(list)
authorCounter = defaultdict(int)
for k,v in trainData.items():
    for author in v["author"]:
        authorIndexListDict[author].append(k)
        authorCounter[author] += 1
print("min freq author happen " + str(min(authorCounter, key=authorCounter.get)) + " times")

min freq author happen 2217 times


In [4]:
def becomeFalse (dict):
    allAuthor = list(range(2032))
    authors = copy.deepcopy(dict["author"])
    authors.append(dict["target"])
    complement = [author for author in allAuthor if author not in authors]
    [temp] = random.sample(complement, 1)
    dict["target"] = temp


randomAuthorsList2000 = random.sample(range(0, 2032), 2000)

devData = {}
for author in randomAuthorsList2000:
    while(True):
        [sampleChoosen] = random.sample(authorIndexListDict[author], 1)
        if not sampleChoosen in devData.keys():
            devData[sampleChoosen] = trainData[sampleChoosen]
            devData[sampleChoosen]["target"] = author
            devData[sampleChoosen]["TrueTarget"] = author
            devData[sampleChoosen]["author"].remove(author)
            trainData.pop(sampleChoosen)
            break
            
len(trainData.items())
falseDevdataKey = random.sample(list(devData.keys()) , 1000)
for k in falseDevdataKey:
    becomeFalse(devData[k])
for k, v in devData.items():
    v["coauthor"] = v.pop("author")
    if v["target"] == v["TrueTarget"]:
        v["label"] = 1
    else:
        v["label"] = 0

In [5]:
print("Develop set is devData, it has " + str(len(devData)) + " papers")
for k, v in devData.items():
    print(v.keys())
    break
print("train set is trainData, it has " + str(len(trainData)) + " papers")

Develop set is devData, it has 2000 papers
dict_keys(['venue', 'keywords', 'year', 'target', 'TrueTarget', 'coauthor', 'label'])
train set is trainData, it has 24108 papers


In [36]:
def add_keyword_list(keyword_index):
    list = [0] * 500
    for item in keyword_index:
        list[item] = 1
    return list

def add_author_list(author_index):
    list = [0] * 2302
    for item in author_index:
        list[item] = 1
    return list

trainData_list = []  
for k,v in trainData.items():
    row_list = []
    row_list.append(v["year"])
    if v["venue"] == "":
        row_list.append(470)
    else:
        row_list.append(v["venue"])
    row_list.extend(add_keyword_list(v["keywords"]))
    row_list.extend(add_author_list(v["author"]))
    trainData_list.append(row_list)

print(len(trainData_list), len(trainData_list[0]))  

testData_list = []  
for k,v in testData.items():
    row_list = []
    row_list.append(v["year"])
    if v["venue"] == "":
        row_list.append(470)
    else:
        row_list.append(v["venue"])
    row_list.extend(add_keyword_list(v["keywords"]))
    row_list.extend(add_author_list(v["coauthor"]))
    row_list.append(v["target"])
    testData_list.append(row_list)

devData_list = []  
for k,v in devData.items():
    row_list = []
    row_list.append(v["year"])
    if v["venue"] == "":
        row_list.append(470)
    else:
        row_list.append(v["venue"])
    row_list.extend(add_keyword_list(v["keywords"]))
    row_list.extend(add_author_list(v["coauthor"]))
    row_list.append(v["target"])
    row_list.append(v["TrueTarget"])
    devData_list.append(row_list)

24108 2804


In [42]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from scipy import sparse
from sklearn import metrics

In [8]:
clf = OneVsRestClassifier(LogisticRegression(), n_jobs=-1)

In [16]:
X_train_transform = sparse.csr_matrix(np.array([row[0:502] for row in trainData_list]))
y_train = sparse.csr_matrix(np.array([row[502:] for row in trainData_list]))

In [21]:
clf.fit(X_train_transform, y_train)

OneVsRestClassifier(estimator=LogisticRegression(), n_jobs=-1)

In [22]:
X_dev_transform = np.array([row[0:502] for row in devData_list])

In [23]:
y_dev_predict_proba_all = clf.predict_proba(X_dev_transform)

In [44]:
targets = [row[2804] for row in devData_list]
true_targets_author = [row[2805] for row in devData_list]
true_targets = []
for i in range(len(true_targets_author)):
    if true_targets_author[i] == targets[i]:
        true_targets.append(1)
    else:
        true_targets.append(0)

In [40]:
targets_proba = []
row_counter = 0
for item in targets:
    targets_proba.append(y_dev_predict_proba_all[row_counter][item])
    row_counter += 1   

In [46]:
fpr, tpr, thresholds = metrics.roc_curve(true_targets, targets_proba, pos_label=1)
print(metrics.auc(fpr, tpr))

0.604079


In [48]:
# Openinng the JSON file
openTrainData = open('data/train.json');
openTestData = open('data/test.json');

# returns JSON object as
# a dictionary
trainData = json.load(openTrainData);
testData = json.load(openTestData);

# Iterating through the trainData
#for i in trainData:
#    print(trainData[i]);

# Iterating through the testData
#for i in testData:
#    print(testData[i]);

# Closing file
openTrainData.close();
openTestData.close();

len(trainData.items())

26108

In [49]:
trainData_list = []  
for k,v in trainData.items():
    row_list = []
    row_list.append(v["year"])
    if v["venue"] == "":
        row_list.append(470)
    else:
        row_list.append(v["venue"])
    row_list.extend(add_keyword_list(v["keywords"]))
    row_list.extend(add_author_list(v["author"]))
    trainData_list.append(row_list)
print(len(trainData_list), len(trainData_list[0]))  

26108 2804


In [50]:
clf = OneVsRestClassifier(LogisticRegression(), n_jobs=-1)

In [57]:
X_train_transform = sparse.csr_matrix(np.array([row[2:502] for row in trainData_list]))
y_train = sparse.csr_matrix(np.array([row[502:] for row in trainData_list]))

In [58]:
clf.fit(X_train_transform, y_train)

OneVsRestClassifier(estimator=LogisticRegression(), n_jobs=-1)

In [60]:
X_test_transform = np.array([row[2:502] for row in testData_list])
y_test_predict_proba_all = clf.predict_proba(X_test_transform)

In [62]:
# targets = [row[2804] for row in testData_list]
# true_targets_author = [row[2805] for row in testData_list]
# true_targets = []
# for i in range(len(true_targets_author)):
#     if true_targets_author[i] == targets[i]:
#         true_targets.append(1)
#     else:
#         true_targets.append(0)

In [63]:
targets_proba = []
row_counter = 0
for item in targets:
    targets_proba.append(y_dev_predict_proba_all[row_counter][item])
    row_counter += 1   

In [64]:
import csv

header = ['Id','Predicted']
data = []
idNumber = 0
for i in targets_proba:
    data.append([idNumber,i])
    idNumber = idNumber+1

filename = 'multi_label_logistic_regression.csv'
with open(filename, 'w', newline="") as file:
    csvwriter = csv.writer(file)
    csvwriter.writerow(header)
    csvwriter.writerows(data)